In [ ]:
!pip install -q torch torchvision torchaudio
!pip install -q transformers==4.25.1 accelerate datasets evaluate diffusers==0.16.1 xformers triton scipy clip gradio

Install fastcomposer from github. The package wasn't published.

In [ ]:
!git clone https://github.com/mit-han-lab/fastcomposer.git
%cd fastcomposer
!python setup.py install
%cd ..

Download pre-trained model.

In [ ]:
!mkdir -p model/fastcomposer
%cd model/fastcomposer
!wget https://huggingface.co/mit-han-lab/fastcomposer/resolve/main/pytorch_model.bin
%cd ../..

Define required arguments.

In [ ]:
CAPTION="a man <|image|> and a man <|image|> are reading book together"
DEMO_NAME="newton_einstein"


from dataclasses import dataclass


@dataclass()
class Args:
    adam_beta1=0.9
    adam_beta2=0.999
    adam_epsilon=1e-08
    adam_weight_decay=0.01
    allow_tf32=False
    balance_num_objects=False
    cache_dir=None
    caption_column='caption'
    center_crop=False
    checkpointing_steps=500
    dataloader_num_workers=0
    dataset_config_name=None
    dataset_format='huggingface'
    dataset_name=None
    dataset_name1=None
    dataset_name2=None
    dataset_name3=None
    dataset_type='original'
    dataset_type1='original'
    dataset_type2='original'
    dataset_type3='original'
    disable_flashattention=False
    enable_xformers_memory_efficient_attention=False
    end_idx=50
    evaluation_batch_size=4
    finetuned_model_path='model/fastcomposer/pytorch_model.bin'
    freeze_unet=False
    generate_height=512
    generate_width=512
    gradient_accumulation_steps=1
    gradient_checkpointing=False
    guidance_scale=5
    hub_model_id=None
    hub_token=None
    image_column='image'
    image_encoder_name_or_path='openai/clip-vit-large-patch14'
    image_encoder_trainable_layers=0
    image_encoder_type='clip'
    image_encoder_use_lora=False
    inference_split='eval'
    inference_steps=50
    keep_interval=None
    keep_only_last_checkpoint=False
    learning_rate=0.0001
    load_merged_lora_model=False
    load_model=None
    local_rank=-1
    localization_layers=5
    logging_dir='logs'
    lora_image_encoder_alpha=16
    lora_image_encoder_bias='none'
    lora_image_encoder_dropout=0.1
    lora_image_encoder_r=16
    lora_text_encoder_alpha=16
    lora_text_encoder_bias='none'
    lora_text_encoder_dropout=0.1
    lora_text_encoder_r=16
    lr_scheduler='constant'
    lr_warmup_steps=500
    mask_loss=False
    mask_loss_prob=0.5
    max_grad_norm=1.0
    max_num_objects=4
    max_train_samples=None
    max_train_steps=None
    min_num_objects=None
    mixed_precision='fp16'
    model_type='dreamer'
    no_object_augmentation=False
    non_ema_revision=None
    num_batches=1
    num_datasets=1
    num_image_tokens=1
    num_images_per_prompt=1
    num_rows=1
    num_train_epochs=100
    object_appear_prob=1
    object_background_processor='random'
    object_localization=False
    object_localization_loss='balanced_l1'
    object_localization_normalize=False
    object_localization_skip_special_tokens=False
    object_localization_threshold=1.0
    object_localization_weight=0.01
    object_resolution=256
    object_types=None
    output_dir='log/fine_generator'
    pretrained_model_name_or_path='runwayml/stable-diffusion-v1-5'
    push_to_hub=False
    random_flip=False
    report_to=None
    resume_from_checkpoint=None
    retrieval_identity_path=None
    retrieval_identity_path1=None
    retrieval_identity_path2=None
    retrieval_identity_path3=None
    revision=None
    scale_lr=False
    seed=None
    start_idx=0
    start_merge_step=0
    test_caption=None
    test_reference_folder=None
    test_resolution=512
    text_encoder_use_lora=False
    text_only_prob=0
    text_prompt_only=False
    train_batch_size=16
    train_data_dir=None
    train_image_encoder=False
    train_resolution=256
    train_text_encoder=False
    uncondition_prob=0
    unet_lora_alpha=1.0
    unet_lr_scale=1.0
    unet_use_lora=False
    use_8bit_adam=False
    use_ema=False
    use_multiple_conditioning=False
    use_multiple_datasets=False
    val_dataset_name=None

Initialise model. After notebook restarting you should remove one extra `fastcomposer` from import path.

In [ ]:
from fastcomposer.fastcomposer.model import FastComposerModel


model = FastComposerModel.from_pretrained(Args)

Convert image encoder that cause error "No conversion rule found for operations: aten::_upsample_bilinear2d_aa, aten::sub_"

In [ ]:
from openvino.runtime import PartialShape
from openvino.tools import mo
from openvino.tools.mo.convert import InputCutInfo


image_encoder_ir = mo.convert_model(
    model.image_encoder,
    input=[InputCutInfo(shape=PartialShape([1,2,3,256,256]), type='f32')],
)